<img align="centre" src="../../Supplementary_data/Github_banner.jpg" width="100%">

# Enhanced Combined Drought Index - Vegetation Drought Index

* **Products used:** 
[ls5_sr](https://explorer.digitalearth.africa/products/ls5_sr),
[ls7_sr](https://explorer.digitalearth.africa/products/ls7_sr),
[ls8_sr](https://explorer.digitalearth.africa/products/ls8_sr),
[ls9_sr](https://explorer.digitalearth.africa/products/ls9_st)

## Background 

Drought is an extended period, during which, fresh water availability and accessibility for the ecosystem at a given time and place is below normal, due to unfavourable spatial and temporal distribution of rainfall, temperature, soil moisture and wind characteristics [(Balint et al., 2013)](https://doi.org/10.1016/B978-0-444-59559-1.00023-2). Severe droughts can affect large populations, leading to a long-term threat to people’s livelihoods and result in tremendous economic loss [(Enenkel et al., 2016)](https://doi.org/10.3390/rs8040340).

The **Enhanced Drought Index** is a combination of the following components: Vegetation, Precipitation, Temperature and Soil moisture.

This notebook will focus on the **Vegetation component**, which considers Normalized Difference Vegetation Index (NDVI) deficits and the persistence of these deficits. The index based on the vegetation component is called the Vegetation Drought Index (VDI)." The Vegetation Drought Index (VDI) is a remote sensing-based indicator designed to assess the impact of drought on vegetation health. It combines data from satellite imagery, specifically from vegetation indices like the Normalized Difference Vegetation Index (NDVI), and incorporates factors related to drought conditions, such as moisture stress and temperature anomalies. The VDI provides a way to monitor the health of vegetation under drought stress over large areas and is useful in agricultural, forestry, and environmental monitoring


The notebook outlines:

***
1. Loading Python and DE Africa Packages
2. Setting dask cluster
3. Loading area on interest
5. Load Landsat Surface reflectance and Calculate the Normalized Difference Vegetation Index (NDVI)
6. Resampling of data to dekadal (10-day) intervals
7. Calculate the Vegetation Drought Index
8. Export the result as zip

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages

In [1]:
import json
import os

import dask
import geopandas as gpd
import numpy as np
import pandas as pd
import rioxarray
import toolz
import xarray as xr
import datacube
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.datahandling import load_ard
from dekad import group_by_dekad, bin_by_interest_period, get_dekad_no_in_year, get_no_data_mask, max_consecutive_ones
from odc.geo.geobox import GeoBox
from odc.geo.geom import Geometry
from odc.geo.xr import rasterize
from pyarrow import Table
from pyarrow.parquet import write_table

### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [2]:
# Connect to the datacube
dc = datacube.Datacube(app="VegetationDroughtIndex")

### Set up a Dask cluster

Dask can be used to better manage memory use and conduct the analysis in parallel. 
For an introduction to using Dask with Digital Earth Africa, see the [Dask notebook](../../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in DE Africa* section of the [Dask notebook](../../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

To use Dask, set up the local computing cluster using the cell below.

In [3]:
create_local_dask_cluster()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/victoria@kartoza.com/proxy/8787/status,
Dashboard: /user/victoria@kartoza.com/proxy/8787/status,Workers: 1
Total threads: 4,Total memory: 26.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38165,Workers: 1
Dashboard: /user/victoria@kartoza.com/proxy/8787/status,Total threads: 4
Started: Just now,Total memory: 26.21 GiB
Comm: tcp://127.0.0.1:44311,Total threads: 4
Dashboard: /user/victoria@kartoza.com/proxy/33787/status,Memory: 26.21 GiB
Nanny: tcp://127.0.0.1:34551,


### Analysis parameters

The following cell sets important parameters for the analysis:

* `country`: The country to run the analysis over.
* `resolution`: The x and y cell resolution of the satellite data in metres (spatial resolution). We'll use 5,000 m, which is approximately equal to the default CHIRPS resolution.
* `output_crs` : The coordinate reference system that the loaded data is to be reprojected to.
* `dask_chunks`:  the size of the dask chunks, dask breaks data into manageable chunks that can be easily stored in memory, e.g. `dict(x=1000,y=1000)`
* `time_range` : Time range to load data for.
* `ip` : The interest period to use to calculate the drought indices e.g. (3, 4, 5 dekads). It defines to what extent past observations are considered. Longer IPs detect more severe drought events. For example, if the IP=3 dekads, then the drought index (say PDI) of 0.35 for dekad 2 of 2006 implies actual drought for dekad 36 of 2005, dekad 1 of 2006 and dekad 2 of 2006.
* `output_dir` :  The directory in which to store results from the analysis.

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.

In [4]:
resolution = (-5000, 5000)
output_crs = "EPSG:6933"
dask_chunks = dict(x=100,y=100)
time_range = ("2014","2025")
output_dir = "results"

# Corresponding to the six-months Standardized Precipitation Evapotranspiration Index
ip = 18

# Create the outpur directory if it does not exist.
os.makedirs(output_dir, exist_ok=True)

In [5]:
#loading the countries shapefiles
african_countries_url = "https://raw.githubusercontent.com/digitalearthafrica/deafrica-sandbox-notebooks/main/Supplementary_data/Rainfall_anomaly_CHIRPS/african_countries.geojson"
african_countries = gpd.read_file(african_countries_url)
print(np.unique(african_countries.COUNTRY))

['Algeria' 'Angola' 'Benin' 'Botswana' 'Burkina Faso' 'Burundi' 'Cameroon'
 'Cape Verde' 'Central African Republic' 'Chad' 'Comoros'
 'Congo-Brazzaville' 'Cote d`Ivoire' 'Democratic Republic of Congo'
 'Djibouti' 'Egypt' 'Equatorial Guinea' 'Eritrea' 'Ethiopia' 'Gabon'
 'Gambia' 'Ghana' 'Guinea' 'Guinea-Bissau' 'Kenya' 'Lesotho' 'Liberia'
 'Libya' 'Madagascar' 'Malawi' 'Mali' 'Mauritania' 'Morocco' 'Mozambique'
 'Namibia' 'Niger' 'Nigeria' 'Rwanda' 'Sao Tome and Principe' 'Senegal'
 'Sierra Leone' 'Somalia' 'South Africa' 'Sudan' 'Swaziland' 'Tanzania'
 'Togo' 'Tunisia' 'Uganda' 'Western Sahara' 'Zambia' 'Zimbabwe']


In [6]:
# Select the country of interest by providing the name below
country = "Ethiopia"
idx = african_countries[african_countries['COUNTRY'] == country].index[0]
geopolygon = Geometry(geom=african_countries.iloc[idx].geometry, crs=african_countries.crs)

# Create the output folder for the selected country
output_path = os.path.join(output_dir, f"VDI_{country}.parquet")

### Load Landsat surface reflectance data
Load the Landsat surface reflectance data from the datacube using the analysis parameters set in the previous section and the measurements used to calculate the **NDVI** spectral index.

In [7]:
%%time
# Load the Landsat data
ds_ls_sr = load_ard(dc=dc,
                    products=["ls8_sr", "ls9_sr"],
                    measurements=["nir", "red"],
                    min_gooddata=0.9,
                    time=time_range,
                    dask_chunks=dict(time=366),
                    geopolygon=geopolygon,
                    resolution=resolution,
                    output_crs=output_crs,
                )

ds_ls_sr

Using pixel quality parameters for USGS Collection 2
Finding datasets
    ls8_sr


/home/jovyan/dev/deafrica-sandbox-notebooks/Tools/deafrica_tools/datahandling.py:244: UserWarning: Setting 'min_gooddata' percentage to > 0.0 will cause dask arrays to compute when loading pixel-quality data to calculate 'good pixel' percentage. This can slow the return of your dataset.
  warnings.warn(


    ls9_sr
Counting good quality pixels for each time step


/opt/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 71.60 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/opt/venv/lib/python3.12/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


Filtering to 20393 out of 20393 time steps with at least 90.0% good quality pixels
Applying pixel quality/cloud mask
Re-scaling Landsat C2 data
Returning 20393 time steps as a dask array
CPU times: user 3min 9s, sys: 8.32 s, total: 3min 18s
Wall time: 24min 49s


<xarray.Dataset> Size: 14GB
Dimensions:      (time: 20393, y: 291, x: 290)
Coordinates:
  * time         (time) datetime64[ns] 163kB 2014-01-02T07:40:11.005523 ... 2...
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
Data variables:
    nir          (time, y, x) float32 7GB dask.array<chunksize=(366, 291, 290), meta=np.ndarray>
    red          (time, y, x) float32 7GB dask.array<chunksize=(366, 291, 290), meta=np.ndarray>
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

### Masked the landsat data

In [8]:
# Rasterize the country geopolygon and mask the loaded  Landsat data.
country_mask = rasterize(poly=geopolygon, how=ds_ls_sr.odc.geobox)
ds_ls_sr = ds_ls_sr.where(country_mask)

### Calculate NDVI index

In [9]:
# Calculate the NDVI spectral index using the calculate indices.
ds_ndvi = calculate_indices(ds_ls_sr, index=['NDVI'], satellite_mission='ls', drop=True)
# Convert to DataArray
da_ndvi = ds_ndvi["NDVI"]

da_ndvi

Dropping bands ['nir', 'red']


<xarray.DataArray 'NDVI' (time: 20393, y: 291, x: 290)> Size: 7GB
dask.array<truediv, shape=(20393, 291, 290), dtype=float32, chunksize=(366, 291, 290), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 163kB 2014-01-02T07:40:11.005523 ... 2...
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933

## Resample data

Resample the NDVI `da_ndvi` timeseries into **dekadal** (10-day) timesteps.

In [10]:
%%time
# Resample the NDVI data from daily to decadal (10-day) intervals
da_ndvi_dekadal = group_by_dekad(da_ndvi).mean(dim="time").compute()
da_ndvi_dekadal

/opt/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 76.79 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 3min 15s, sys: 16.4 s, total: 3min 31s
Wall time: 1h 14min 47s


<xarray.DataArray 'NDVI' (dekad: 403, y: 291, x: 290)> Size: 136MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-03-11

### Modify the timeseries

The raw time series of NDVI are modified to adjust the range of all variables and avoid division by zero.

In [11]:
da_ndvi_dekadal_modified =  da_ndvi_dekadal - (da_ndvi_dekadal.min() - 0.01)

### Group the modified timeseries using the `ip` parameter

The `ip` parameter determines to what extent past observations are considered. Longer IPs detect more severe drought events. The default `ip` used of **18 dekads** corresponds to the 6-month Standardized Precipitation Evapotranspiration Index.
For example, calculating the Vegetation Drought Index for the dekad `2011-01-11` using an interest period of `ip=3` requires vegetation data from the dekad `2010-12-21`, `2011-01-01` and `2011-01-11`.


**Each interest period is labelled using its end dekad.**

In [12]:
# Group the modified NDVI dekadal timeseries by the interest period.
# The dictionary keys are the end dekad for each interest period.
# The values of the dictionary are the NDVI data that belongs to the interest period
da_ndvi_binned_by_interest_period = bin_by_interest_period(ds=da_ndvi_dekadal_modified, ip=ip)

# Get the interest periods
interest_periods = list(da_ndvi_binned_by_interest_period.keys())

### Get the average values for each interest period

Get the actual average NDVI for each interest period.

In [13]:
da_ndvi_actual_avg_for_ip = xr.concat([da_ndvi_binned_by_interest_period[i].mean(dim="dekad").assign_coords(dekad=i).expand_dims({"dekad": 1}) for i in interest_periods], dim="dekad")
display(da_ndvi_actual_avg_for_ip)

<xarray.DataArray 'NDVI' (dekad: 404, y: 291, x: 290)> Size: 273MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-03-11

### Get the long term average for each interest period over the years of available data


In [14]:
# Group the interest periods by year by getting the dekad number in the year for the end dekad of the interest period. 
grouped_by_year = toolz.groupby(lambda dekad: get_dekad_no_in_year(date=dekad), interest_periods)

#Get the long term average NDVI for each interest period.
long_term_avg_ndvi = {}
for dekad_no_in_year, interest_periods_list in grouped_by_year.items():
    long_term_avg_for_period = xr.concat([da_ndvi_binned_by_interest_period[i] for i in interest_periods_list], dim="dekad").mean(dim="dekad")
    long_term_avg_ndvi[dekad_no_in_year] = long_term_avg_for_period
    
da_ndvi_long_term_avg_for_ip = xr.concat([long_term_avg_ndvi[get_dekad_no_in_year(i)].assign_coords(dekad=i).expand_dims({"dekad": 1}) for  i in interest_periods], dim="dekad")

assert all(da_ndvi_actual_avg_for_ip.dekad.values == da_ndvi_long_term_avg_for_ip.dekad.values)

da_ndvi_long_term_avg_for_ip

<xarray.DataArray 'NDVI' (dekad: 404, y: 291, x: 290)> Size: 273MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-03-11

## Get the actual length of continous deficit in each interest period

For NDVI, the run length is the number of successive dekads during an interest period that are below the long-term average for the same period.

In [15]:
%%time
# For each interest period, determine the maximum number of successive dekads with NDVI below the long-term average for that period.
ndvi_run_length = []
for interest_period in interest_periods:
    # Get the dekads in the interest period
    ds = da_ndvi_binned_by_interest_period[interest_period]
    
    # Identify pixels which are empty for all dekads
    no_data_mask = xr.apply_ufunc(
        get_no_data_mask,
        ds,
        input_core_dims=[["dekad"]],
        vectorize=True,
        dask="allowed",
    )
    
    # Get the long term average for the interest period.
    long_term_avg = da_ndvi_long_term_avg_for_ip.sel(dekad=interest_period)
    
    # Get the pixels where the NDVI is below the long term average NDVI
    mask = xr.where(ds < long_term_avg, 1, 0)
    
    # Get the maximum number of successive dekads below long term average NDVI.
    actual_run_length = xr.apply_ufunc(
        max_consecutive_ones,
        mask,
        input_core_dims=[["dekad"]],
        vectorize=True,
        dask="allowed",
    )
    
    # Modify the run length
    modified_run_length = (actual_run_length.max() + 1) - actual_run_length
    modified_run_length = modified_run_length.where(~no_data_mask)
    ndvi_run_length.append(modified_run_length.assign_coords(dekad=interest_period).expand_dims({"dekad": 1}))

da_ndvi_actual_run_length = xr.concat(ndvi_run_length, dim="dekad")
da_ndvi_actual_run_length

CPU times: user 6min 5s, sys: 3.27 s, total: 6min 8s
Wall time: 6min 4s


<xarray.DataArray 'NDVI' (dekad: 404, y: 291, x: 290)> Size: 273MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    spatial_ref  int32 4B 6933
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-03-11

### Get the long term average of continous deficit in each interest period

In [16]:
# Get the long term average NDVI run length for each interest period 
long_term_avg_run_lenth_ndvi = {}
for dekad_no_in_year, interest_periods_list in grouped_by_year.items():
    long_term_avg_for_period = da_ndvi_actual_run_length.sel(dekad=interest_periods_list).mean(dim="dekad")
    long_term_avg_run_lenth_ndvi[dekad_no_in_year] = long_term_avg_for_period
    
da_ndvi_long_term_avg_run_length_for_ip = xr.concat([long_term_avg_run_lenth_ndvi[get_dekad_no_in_year(i)].assign_coords(dekad=i).expand_dims({"dekad": 1}) for  i in interest_periods], dim="dekad")
da_ndvi_long_term_avg_run_length_for_ip

<xarray.DataArray 'NDVI' (dekad: 404, y: 291, x: 290)> Size: 273MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
    spatial_ref  int32 4B 6933
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-03-11

### Calculate the Vegetation Drought Index (VDI)

In [17]:
# Using the VDI formulae below
VDI = (da_ndvi_actual_avg_for_ip / da_ndvi_long_term_avg_for_ip) * np.sqrt((da_ndvi_actual_run_length / da_ndvi_long_term_avg_run_length_for_ip))
VDI

<xarray.DataArray 'NDVI' (dekad: 404, y: 291, x: 290)> Size: 273MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-03-11

In [18]:
#Scale the results of the VDI
VDI_scaled = (VDI - VDI.min()) / (VDI.max() - VDI.min())
VDI_scaled

<xarray.DataArray 'NDVI' (dekad: 404, y: 291, x: 290)> Size: 273MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * y            (y) float64 2kB 1.882e+06 1.878e+06 ... 4.375e+05 4.325e+05
  * x            (x) float64 2kB 3.182e+06 3.188e+06 ... 4.622e+06 4.628e+06
    spatial_ref  int32 4B 6933
  * dekad        (dekad) datetime64[ns] 3kB 2014-01-01 2014-01-11 ... 2025-03-11

In [19]:
# Convert to DataFrame.
df = VDI_scaled.to_dataframe().drop(columns="spatial_ref")
df

NDVI
dekad      y         x              
2014-01-01 1882500.0 3182500.0   NaN
                     3187500.0   NaN
                     3192500.0   NaN
                     3197500.0   NaN
                     3202500.0   NaN
...                              ...
2025-03-11 432500.0  4607500.0   NaN
                     4612500.0   NaN
                     4617500.0   NaN
                     4622500.0   NaN
                     4627500.0   NaN

[34093560 rows x 1 columns]

In [20]:
# Convert the DataFrame to an Arrow table
table = Table.from_pandas(df)

# Get the tables existing metadata
existing_meta = table.schema.metadata

# Dump the crs of the DataArray as new metadata to JSON.
meta_json = json.dumps(dict(crs=str(VDI_scaled.rio.crs)))

# Combine the metadata
combined_meta = {b"xarray.metadata": meta_json.encode(),**existing_meta}

# Replace the metadata.
table = table.replace_schema_metadata(combined_meta)

### Export the results in a Zip file

In [21]:
# Write the table
write_table(table, output_path, compression="GZIP")

---

## Additional information

<b> License </b> The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0).

Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

<b> Contact </b> If you need assistance, please post a question on the [DE Africa Slack channel](https://digitalearthafrica.slack.com/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).

If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

<b> Compatible datacube version :</b>

In [22]:
print(datacube.__version__)

1.8.20


**Last Tested:**

In [23]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2025-03-20'